In [ ]:
# Colabis kasutamiseks
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/thesis/')

!pip install estnltk==1.7.4
!pip install evaluate
!pip install seqeval
!pip install nervaluate

In [2]:
from modules.data_processing import DatasetProcessor
from modules.bert_data_processing import BERTDataProcessor
from modules.bert_evaluator import BERTEvaluator
from modules.bert_trainer import BERTTrainer
import json
ALL_TAGS = DatasetProcessor.ALL_TAGS
TAG2IDX = DatasetProcessor.TAG2IDX
IDX2TAG = DatasetProcessor.IDX2TAG

INFO:datasets:PyTorch version 2.5.1+cu124 available.
INFO:datasets:Polars version 1.21.0 available.
INFO:datasets:Duckdb version 1.1.3 available.
INFO:datasets:TensorFlow version 2.18.0 available.
INFO:datasets:JAX version 0.4.33 available.


INFO:config.py:54: PyTorch version 2.5.1+cu124 available.
INFO:config.py:66: Polars version 1.21.0 available.
INFO:config.py:77: Duckdb version 1.1.3 available.
INFO:config.py:112: TensorFlow version 2.18.0 available.
INFO:config.py:125: JAX version 0.4.33 available.


In [6]:
def train_model(model_name, dataset_name, epochs=3, batch_size=16):
    if dataset_name.lower() == 'combined':
      ewt_processor = DatasetProcessor('ewt', from_json=True)
      edt_processor = DatasetProcessor('edt', from_json=True)
      ewt_dataset = DatasetProcessor.tag_to_id(ewt_processor.dataset, TAG2IDX)
      edt_dataset = DatasetProcessor.tag_to_id(edt_processor.dataset, TAG2IDX)
      dataset = DatasetProcessor.combine_datasetdicts(ewt_dataset, edt_dataset)
    elif dataset_name.lower() in ['ewt', 'edt']:
      processor = DatasetProcessor(dataset_name.lower(), from_json=True)
      dataset = DatasetProcessor.tag_to_id(processor.dataset, TAG2IDX)

    print(f'{dataset_name.upper()} andmestik laetud')
    bert_processor = BERTDataProcessor(model_name)
    evaluator = BERTEvaluator(all_tags=ALL_TAGS)

    tokenized_dataset = bert_processor.tokenize_dataset(dataset)

    trainer = BERTTrainer(model_name=model_name, idx2tag=IDX2TAG, tag2idx=TAG2IDX, evaluator=evaluator)

    model, model_trainer = trainer.finetune_model(processor=bert_processor, tokenized_dataset=tokenized_dataset, epochs=epochs, batch_size=batch_size, early_stop_patience=3, output_dir=f'./results/{model_name.split("/")[1]}/{dataset_name}/trained_for_{epochs}')

    results = evaluator.evaluate_and_print(tokenized_dataset['test'], model_trainer)
    evaluator.evaluation_to_json(nervaluate_strict_overall=results[1]['strict'], nervaluate_by_tag=results[2], model_name=model_name.split('/')[1], trained_on=dataset_name, evaluated_on=dataset_name, epochs=epochs)

    if dataset_name.lower() == 'combined':
      tokenized_ewt = bert_processor.tokenize_dataset(ewt_dataset['test'])
      tokenized_edt = bert_processor.tokenize_dataset(edt_dataset['test'])

      print(f'Kombineeritud andmestikul treenitud {model_name} EWT testandmestikul')
      ewt_results = evaluator.evaluate_and_print(tokenized_ewt, model_trainer)
      evaluator.evaluation_to_json(nervaluate_strict_overall=ewt_results[1]['strict'], nervaluate_by_tag=ewt_results[2], model_name=model_name.split('/')[1], trained_on=dataset_name, evaluated_on="EWT", epochs=epochs)

      print(f'Kombineeritud andmestikul treenitud {model_name} EDT testandmestikul')
      edt_results = evaluator.evaluate_and_print(tokenized_edt, model_trainer)
      evaluator.evaluation_to_json(nervaluate_strict_overall=edt_results[1]['strict'], nervaluate_by_tag=edt_results[2], model_name=model_name.split('/')[1], trained_on=dataset_name, evaluated_on="EDT", epochs=epochs)

      return model, model_trainer, (results, ewt_results, edt_results)

    return model, model_trainer, results

def train_all():
  dataset_names = ['ewt', 'edt', 'combined']

  model_names = ["tartuNLP/EstRoBERTa", "tartuNLP/EstBERT"]
  results_dict = {}

  for model_name in model_names:
    model_results = {}
    for dataset_name in dataset_names:
      model, model_trainer, results = train_model(model_name, dataset_name, epochs=3)
      model_results[dataset_name] = results
    results_dict[model_name] = model_results

  return results_dict

In [ ]:
def main():
    results = train_all()

In [ ]:
main()

EWT andmestik laetud


tokenizer_config.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

...

Map:   0%|          | 0/3122 [00:00<?, ? examples/s]

Map:   0%|          | 0/3207 [00:00<?, ? examples/s]

Kombineeritud andmestikul treenitud tartuNLP/EstBERT EWT testandmestikul
Hindan testandmestikul..


Seqeval tulemused
EVE {'precision': 0.5517241379310345, 'recall': 0.5614035087719298, 'f1': 0.5565217391304348, 'number': 57}
GEP {'precision': 0.2631578947368421, 'recall': 0.7272727272727273, 'f1': 0.3864734299516908, 'number': 55}
LOC {'precision': 0.4864864864864865, 'recall': 0.6545454545454545, 'f1': 0.5581395348837209, 'number': 55}
ORG {'precision': 0.7235294117647059, 'recall': 0.5324675324675324, 'f1': 0.6134663341645885, 'number': 231}
PER {'precision': 0.9608879492600423, 'recall': 0.8301369863013699, 'f1': 0.8907398334149927, 'number': 1095}
PROD {'precision': 0.5, 'recall': 0.34806629834254144, 'f1': 0.41042345276872966, 'number': 362}
overall_precision 0.7663438256658596
overall_recall 0.6824797843665769
overall_f1 0.7219846022241232
overall_accuracy 0.9505855469865753

Nervaluate tulemused
Strict {'correct': 1266, 'incorrect': 260, 'partial': 0, 'missed': 329, 'spurious': 137, 'possible': 1855, 'actual': 1663, 'precision': 0.7612748045700541, 'recall': 0.682479784366576

Seqeval tulemused
EVE {'precision': 0.5614035087719298, 'recall': 0.38095238095238093, 'f1': 0.45390070921985815, 'number': 84}
GEP {'precision': 0.6736596736596736, 'recall': 0.7526041666666666, 'f1': 0.7109471094710947, 'number': 384}
LOC {'precision': 0.78125, 'recall': 0.7369614512471655, 'f1': 0.7584597432905484, 'number': 441}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 6}
ORG {'precision': 0.7414030261348006, 'recall': 0.7393689986282579, 'f1': 0.7403846153846154, 'number': 729}
PER {'precision': 0.9294478527607362, 'recall': 0.9114740008594757, 'f1': 0.9203731829030157, 'number': 2327}
PROD {'precision': 0.491679273827534, 'recall': 0.5508474576271186, 'f1': 0.5195843325339727, 'number': 590}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}
overall_precision 0.7941819772528433
overall_recall 0.7957484111330265
overall_f1 0.7949644225506294
overall_accuracy 0.9759567818835686

Nervaluate tulemused
Strict {'correct': 3631, 'incorrect': 694, 'partial': 

In [ ]:
estroberta_edt, estroberta_edt_trainer, estroberta_edt_results = train_model('tartuNLP/EstRoBERTa', 'edt', epochs=6)

EDT andmestik laetud


Map:   0%|          | 0/24601 [00:00<?, ? examples/s]

Map:   0%|          | 0/3122 [00:00<?, ? examples/s]

Map:   0%|          | 0/3207 [00:00<?, ? examples/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at tartuNLP/EstRoBERTa and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Eve precision,Eve recall,Eve f1,Eve number,Gep precision,Gep recall,Gep f1,Gep number,Loc precision,Loc recall,Loc f1,Loc number,Muu precision,Muu recall,Muu f1,Muu number,Org precision,Org recall,Org f1,Org number,Per precision,Per recall,Per f1,Per number,Prod precision,Prod recall,Prod f1,Prod number,Unk precision,Unk recall,Unk f1,Unk number
1,0.078200,0.085064,0.725127,0.699046,0.711848,0.976293,0.517857,0.325843,0.400000,89,0.597134,0.730994,0.657318,513,0.790991,0.528916,0.633935,830,0.000000,0.000000,0.000000,9,0.534636,0.765903,0.629707,393,0.870056,0.913405,0.891204,1686,0.470109,0.328273,0.386592,527,0.000000,0.000000,0.000000,40
2,0.047600,0.087196,0.715017,0.717641,0.716327,0.976666,0.500000,0.438202,0.467066,89,0.570461,0.820663,0.673062,513,0.793907,0.533735,0.638329,830,0.000000,0.000000,0.000000,9,0.548043,0.783715,0.645026,393,0.885313,0.897390,0.891311,1686,0.457330,0.396584,0.424797,527,0.000000,0.000000,0.000000,40
3,0.031100,0.091934,0.725700,0.728897,0.727295,0.978049,0.545455,0.471910,0.506024,89,0.568270,0.851852,0.681747,513,0.832402,0.538554,0.653987,830,0.000000,0.000000,0.000000,9,0.545283,0.735369,0.626219,393,0.914442,0.893832,0.904019,1686,0.472426,0.487666,0.479925,527,0.000000,0.000000,0.000000,40
4,0.019400,0.101696,0.741753,0.737216,0.739477,0.978903,0.603175,0.426966,0.500000,89,0.561470,0.863548,0.680492,513,0.787540,0.593976,0.677198,830,0.000000,0.000000,0.000000,9,0.614894,0.735369,0.669757,393,0.909635,0.901542,0.905570,1686,0.519187,0.436433,0.474227,527,0.000000,0.000000,0.000000,40
5,0.009800,0.121099,0.739312,0.736237,0.737771,0.978446,0.592105,0.505618,0.545455,89,0.557464,0.822612,0.664567,513,0.800000,0.636145,0.708725,830,0.000000,0.000000,0.000000,9,0.534602,0.786260,0.636457,393,0.938452,0.877224,0.906806,1686,0.534279,0.428843,0.475789,527,0.000000,0.000000,0.000000,40
6,0.004600,0.125276,0.761704,0.752386,0.757016,0.979420,0.561798,0.561798,0.561798,89,0.602374,0.791423,0.684078,513,0.802053,0.659036,0.723545,830,0.000000,0.000000,0.000000,9,0.601179,0.778626,0.678492,393,0.936914,0.889680,0.912686,1686,0.551867,0.504744,0.527255,527,0.000000,0.000000,0.000000,40


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-sco

[2025-03-05 11:03:30] tartuNLP/EstRoBERTa treenimine lõpetatud
Kokku kulus: 2265.46 sekundit (0.63 tundi)
Hindan testandmestikul..


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Seqeval tulemused
EVE {'precision': 0.5365853658536586, 'recall': 0.4631578947368421, 'f1': 0.4971751412429378, 'number': 95}
GEP {'precision': 0.6844262295081968, 'recall': 0.7106382978723405, 'f1': 0.6972860125260961, 'number': 470}
LOC {'precision': 0.7062818336162988, 'recall': 0.6656, 'f1': 0.685337726523888, 'number': 625}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 8}
ORG {'precision': 0.7357237715803453, 'recall': 0.7578659370725034, 'f1': 0.7466307277628031, 'number': 731}
PER {'precision': 0.9356855739169272, 'recall': 0.9156468531468531, 'f1': 0.9255577645239673, 'number': 2288}
PROD {'precision': 0.5988805970149254, 'recall': 0.5071090047393365, 'f1': 0.5491873396065012, 'number': 633}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}
overall_precision 0.8025586353944563
overall_recall 0.7757625721352019
overall_f1 0.7889331377069796
overall_accuracy 0.9809939686919245

Nervaluate tulemused
Strict {'correct': 3764, 'incorrect': 829, 'partial': 0, 

In [ ]:
estroberta_combined, estroberta_combined_trainer, estroberta_combined_results = train_model('tartuNLP/EstRoBERTa', 'combined', epochs=6)

COMBINED andmestik laetud


Map:   0%|          | 0/30045 [00:00<?, ? examples/s]

Map:   0%|          | 0/3955 [00:00<?, ? examples/s]

Map:   0%|          | 0/4120 [00:00<?, ? examples/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at tartuNLP/EstRoBERTa and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Map:   0%|          | 0/3207 [00:00<?, ? examples/s]

Kombineeritud andmestikul treenitud tartuNLP/EstRoBERTa EWT testandmestikul
Hindan testandmestikul..


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Seqeval tulemused
EVE {'precision': 0.725, 'recall': 0.5272727272727272, 'f1': 0.6105263157894736, 'number': 55}
GEP {'precision': 0.7692307692307693, 'recall': 0.8333333333333334, 'f1': 0.8, 'number': 60}
LOC {'precision': 0.4482758620689655, 'recall': 0.6290322580645161, 'f1': 0.523489932885906, 'number': 62}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
ORG {'precision': 0.5084745762711864, 'recall': 0.5202312138728323, 'f1': 0.5142857142857143, 'number': 173}
PER {'precision': 0.9600456621004566, 'recall': 0.8302073050345509, 'f1': 0.8904182106934887, 'number': 1013}
PROD {'precision': 0.364741641337386, 'recall': 0.3870967741935484, 'f1': 0.37558685446009393, 'number': 310}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
overall_precision 0.7380050505050505
overall_recall 0.698744769874477
overall_f1 0.7178385016886706
overall_accuracy 0.9609631521342575

Nervaluate tulemused
Strict {'correct': 1169, 'incorrect': 272, 'partial': 0, 'missed': 232, 'spu

Seqeval tulemused
EVE {'precision': 0.46835443037974683, 'recall': 0.3894736842105263, 'f1': 0.4252873563218391, 'number': 95}
GEP {'precision': 0.7134268537074149, 'recall': 0.7574468085106383, 'f1': 0.7347781217750258, 'number': 470}
LOC {'precision': 0.7416520210896309, 'recall': 0.6752, 'f1': 0.7068676716917922, 'number': 625}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 8}
ORG {'precision': 0.7636363636363637, 'recall': 0.8043775649794802, 'f1': 0.7834776815456362, 'number': 731}
PER {'precision': 0.9469730446310207, 'recall': 0.9366258741258742, 'f1': 0.9417710393320149, 'number': 2288}
PROD {'precision': 0.6048951048951049, 'recall': 0.5466034755134281, 'f1': 0.5742738589211619, 'number': 633}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}
overall_precision 0.8176470588235294
overall_recall 0.8021434460016488
overall_f1 0.8098210570120682
overall_accuracy 0.9825673534072901

Nervaluate tulemused
Strict {'correct': 3892, 'incorrect': 745, 'partial': 0

In [ ]:
estbert_combined, estbert_combined_trainer, estbert_combined_results = train_model('tartuNLP/EstBERT', 'combined', epochs=6)

COMBINED andmestik laetud


Map:   0%|          | 0/30045 [00:00<?, ? examples/s]

Map:   0%|          | 0/3955 [00:00<?, ? examples/s]

Map:   0%|          | 0/4120 [00:00<?, ? examples/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at tartuNLP/EstBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Map:   0%|          | 0/3207 [00:00<?, ? examples/s]

Kombineeritud andmestikul treenitud tartuNLP/EstBERT EWT testandmestikul
Hindan testandmestikul..


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Seqeval tulemused
EVE {'precision': 0.559322033898305, 'recall': 0.5789473684210527, 'f1': 0.5689655172413793, 'number': 57}
GEP {'precision': 0.5797101449275363, 'recall': 0.7272727272727273, 'f1': 0.6451612903225806, 'number': 55}
LOC {'precision': 0.40476190476190477, 'recall': 0.6181818181818182, 'f1': 0.48920863309352525, 'number': 55}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
ORG {'precision': 0.7668711656441718, 'recall': 0.5411255411255411, 'f1': 0.6345177664974619, 'number': 231}
PER {'precision': 0.9613428280773143, 'recall': 0.863013698630137, 'f1': 0.9095283926852743, 'number': 1095}
PROD {'precision': 0.39862542955326463, 'recall': 0.32044198895027626, 'f1': 0.3552833078101072, 'number': 362}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
overall_precision 0.7798552472858866
overall_recall 0.6970350404312669
overall_f1 0.7361229718189582
overall_accuracy 0.9520137103684662

Nervaluate tulemused
Strict {'correct': 1293, 'incorrect': 237, '

Seqeval tulemused
EVE {'precision': 0.49056603773584906, 'recall': 0.30952380952380953, 'f1': 0.3795620437956205, 'number': 84}
GEP {'precision': 0.7134831460674157, 'recall': 0.6614583333333334, 'f1': 0.6864864864864865, 'number': 384}
LOC {'precision': 0.7045454545454546, 'recall': 0.7732426303854876, 'f1': 0.7372972972972973, 'number': 441}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 6}
ORG {'precision': 0.7323162274618585, 'recall': 0.7242798353909465, 'f1': 0.7282758620689657, 'number': 729}
PER {'precision': 0.9217657342657343, 'recall': 0.9063171465406102, 'f1': 0.9139761646803901, 'number': 2327}
PROD {'precision': 0.4930662557781202, 'recall': 0.5423728813559322, 'f1': 0.5165456012913641, 'number': 590}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}
overall_precision 0.7849934181658622
overall_recall 0.7841332456717072
overall_f1 0.7845630961517377
overall_accuracy 0.974904596636148

Nervaluate tulemused
Strict {'correct': 3578, 'incorrect': 766, 

EstRoBERTa EDT 3 ja 6, combined 3 ja 6; EstBERT combined 3 ja 6 uuesti, sest output_dir oli sama ja alles jäi ainult üks parim variant

In [7]:
estroberta_edt_3, estroberta_edt_3_trainer, estroberta_edt_3_results = train_model('tartuNLP/EstRoBERTa', 'EDT', epochs=3)

Map:   0%|          | 0/24601 [00:00<?, ? examples/s]

Map:   0%|          | 0/3122 [00:00<?, ? examples/s]

Map:   0%|          | 0/3207 [00:00<?, ? examples/s]

EDT andmestik laetud


Map:   0%|          | 0/24601 [00:00<?, ? examples/s]

...

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Eve precision,Eve recall,Eve f1,Eve number,Gep precision,Gep recall,Gep f1,Gep number,Loc precision,Loc recall,Loc f1,Loc number,Muu precision,Muu recall,Muu f1,Muu number,Org precision,Org recall,Org f1,Org number,Per precision,Per recall,Per f1,Per number,Prod precision,Prod recall,Prod f1,Prod number,Unk precision,Unk recall,Unk f1,Unk number
1,0.072300,0.078209,0.725092,0.724737,0.724914,0.977530,0.560976,0.516854,0.538012,89,0.603150,0.746589,0.667247,513,0.795764,0.633735,0.705567,830,0.000000,0.000000,0.000000,9,0.584677,0.737913,0.652418,393,0.841878,0.925267,0.881605,1686,0.438547,0.297913,0.354802,527,0.000000,0.000000,0.000000,40
2,0.041700,0.086422,0.709577,0.726939,0.718153,0.976892,0.375000,0.269663,0.313725,89,0.533578,0.851852,0.656156,513,0.831418,0.522892,0.642012,830,0.000000,0.000000,0.000000,9,0.490879,0.753181,0.594378,393,0.897674,0.915777,0.906635,1686,0.514161,0.447818,0.478702,527,0.000000,0.000000,0.000000,40
3,0.021200,0.086889,0.758995,0.753609,0.756292,0.979960,0.582418,0.595506,0.588889,89,0.582849,0.781676,0.667777,513,0.803807,0.661446,0.725711,830,0.000000,0.000000,0.000000,9,0.643923,0.768448,0.700696,393,0.922007,0.904508,0.913174,1686,0.528541,0.474383,0.500000,527,0.000000,0.000000,0.000000,40


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[2025-03-10 14:29:53] tartuNLP/EstRoBERTa treenimine lõpetatud
Kokku kulus: 1085.24 sekundit (0.30 tundi)
Hindan testandmestikul..


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Seqeval tulemused
EVE {'precision': 0.5797101449275363, 'recall': 0.42105263157894735, 'f1': 0.48780487804878053, 'number': 95}
GEP {'precision': 0.6808510638297872, 'recall': 0.7505330490405118, 'f1': 0.7139959432048683, 'number': 469}
LOC {'precision': 0.729776247848537, 'recall': 0.6784, 'f1': 0.703150912106136, 'number': 625}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 8}
ORG {'precision': 0.7463672391017173, 'recall': 0.7729138166894665, 'f1': 0.7594086021505375, 'number': 731}
PER {'precision': 0.9239130434782609, 'recall': 0.9287587412587412, 'f1': 0.9263295553618135, 'number': 2288}
PROD {'precision': 0.5315985130111525, 'recall': 0.4518167456556082, 'f1': 0.488471391972673, 'number': 633}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}
overall_precision 0.7963040739185216
overall_recall 0.7816944959802102
overall_f1 0.7889316550504526
overall_accuracy 0.9812886968221571

Nervaluate tulemused
Strict {'correct': 3792, 'incorrect': 827, 'partial': 0, 

In [8]:
estroberta_edt_6, estroberta_edt_6_trainer, estroberta_edt_6_results = train_model('tartuNLP/EstRoBERTa', 'EDT', epochs=6)

Map:   0%|          | 0/24601 [00:00<?, ? examples/s]

Map:   0%|          | 0/3122 [00:00<?, ? examples/s]

Map:   0%|          | 0/3207 [00:00<?, ? examples/s]

EDT andmestik laetud


Map:   0%|          | 0/24601 [00:00<?, ? examples/s]

...

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Eve precision,Eve recall,Eve f1,Eve number,Gep precision,Gep recall,Gep f1,Gep number,Loc precision,Loc recall,Loc f1,Loc number,Muu precision,Muu recall,Muu f1,Muu number,Org precision,Org recall,Org f1,Org number,Per precision,Per recall,Per f1,Per number,Prod precision,Prod recall,Prod f1,Prod number,Unk precision,Unk recall,Unk f1,Unk number
1,0.072600,0.074083,0.740477,0.737216,0.738843,0.978119,0.628205,0.550562,0.586826,89,0.610738,0.709552,0.656447,513,0.833333,0.632530,0.719178,830,0.000000,0.000000,0.000000,9,0.607143,0.778626,0.682274,393,0.871837,0.940095,0.904680,1686,0.415350,0.349146,0.379381,527,0.000000,0.000000,0.000000,40
2,0.047200,0.076375,0.732493,0.739662,0.736060,0.978312,0.549020,0.314607,0.400000,89,0.564537,0.844055,0.676562,513,0.806838,0.568675,0.667138,830,0.000000,0.000000,0.000000,9,0.601578,0.776081,0.677778,393,0.897583,0.925267,0.911215,1686,0.469729,0.426945,0.447316,527,0.000000,0.000000,0.000000,40
3,0.029500,0.088824,0.731725,0.734769,0.733244,0.978757,0.640625,0.460674,0.535948,89,0.570068,0.816764,0.671474,513,0.763441,0.598795,0.671168,830,0.000000,0.000000,0.000000,9,0.555556,0.737913,0.633880,393,0.905771,0.912218,0.908983,1686,0.502304,0.413662,0.453694,527,0.000000,0.000000,0.000000,40
4,0.018300,0.094191,0.755672,0.741620,0.748580,0.980032,0.655738,0.449438,0.533333,89,0.558559,0.846004,0.672868,513,0.804207,0.598795,0.686464,830,0.000000,0.000000,0.000000,9,0.650000,0.694656,0.671587,393,0.903880,0.925860,0.914738,1686,0.560794,0.428843,0.486022,527,0.000000,0.000000,0.000000,40
5,0.008400,0.109570,0.746128,0.754343,0.750213,0.979478,0.617021,0.651685,0.633880,89,0.583679,0.822612,0.682848,513,0.795918,0.610843,0.691207,830,0.000000,0.000000,0.000000,9,0.557223,0.755725,0.641469,393,0.933658,0.909846,0.921598,1686,0.537525,0.502846,0.519608,527,0.000000,0.000000,0.000000,40
6,0.003900,0.115811,0.760800,0.754098,0.757434,0.979887,0.648352,0.662921,0.655556,89,0.594993,0.787524,0.677852,513,0.774854,0.638554,0.700132,830,0.000000,0.000000,0.000000,9,0.606122,0.755725,0.672707,393,0.925030,0.907473,0.916168,1686,0.595455,0.497154,0.541882,527,0.000000,0.000000,0.000000,40


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[2025-03-10 15:07:42] tartuNLP/EstRoBERTa treenimine lõpetatud
Kokku kulus: 2192.37 sekundit (0.61 tundi)
Hindan testandmestikul..


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Seqeval tulemused
EVE {'precision': 0.5540540540540541, 'recall': 0.43157894736842106, 'f1': 0.485207100591716, 'number': 95}
GEP {'precision': 0.6802443991853361, 'recall': 0.7121535181236673, 'f1': 0.6958333333333333, 'number': 469}
LOC {'precision': 0.6961414790996785, 'recall': 0.6928, 'f1': 0.6944667201283079, 'number': 625}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 8}
ORG {'precision': 0.7791495198902606, 'recall': 0.7770177838577291, 'f1': 0.7780821917808218, 'number': 731}
PER {'precision': 0.9322409211691762, 'recall': 0.9200174825174825, 'f1': 0.9260888693356797, 'number': 2288}
PROD {'precision': 0.6091743119266055, 'recall': 0.5244865718799369, 'f1': 0.5636672325976231, 'number': 633}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}
overall_precision 0.8080101716465353
overall_recall 0.7860235003092146
overall_f1 0.7968652037617555
overall_accuracy 0.981425525364591

Nervaluate tulemused
Strict {'correct': 3813, 'incorrect': 789, 'partial': 0, 

In [16]:
estroberta_combined_3, estroberta_combined_3_trainer, estroberta_combined_3_results = train_model('tartuNLP/EstRoBERTa', 'Combined', epochs=3)

Map:   0%|          | 0/5444 [00:00<?, ? examples/s]

Map:   0%|          | 0/833 [00:00<?, ? examples/s]

Map:   0%|          | 0/913 [00:00<?, ? examples/s]

Map:   0%|          | 0/24601 [00:00<?, ? examples/s]

Map:   0%|          | 0/3122 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/913 [00:00<?, ? examples/s]

Map:   0%|          | 0/3207 [00:00<?, ? examples/s]

Kombineeritud andmestikul treenitud tartuNLP/EstRoBERTa EWT testandmestikul
Hindan testandmestikul..


Seqeval tulemused
EVE {'precision': 0.813953488372093, 'recall': 0.6363636363636364, 'f1': 0.7142857142857143, 'number': 55}
GEP {'precision': 0.6790123456790124, 'recall': 0.9166666666666666, 'f1': 0.7801418439716312, 'number': 60}
LOC {'precision': 0.4943820224719101, 'recall': 0.7096774193548387, 'f1': 0.5827814569536423, 'number': 62}
ORG {'precision': 0.5059523809523809, 'recall': 0.4913294797687861, 'f1': 0.4985337243401759, 'number': 173}
PER {'precision': 0.9517241379310345, 'recall': 0.8173741362290227, 'f1': 0.8794476898566118, 'number': 1013}
PROD {'precision': 0.4214046822742475, 'recall': 0.4064516129032258, 'f1': 0.41379310344827586, 'number': 310}
overall_precision 0.756774193548387
overall_recall 0.7011356843992828
overall_f1 0.7278932671424139
overall_accuracy 0.9640609322265803

Nervaluate tulemused
Strict {'correct': 1173, 'incorrect': 274, 'partial': 0, 'missed': 226, 'spurious': 110, 'possible': 1673, 'actual': 1557, 'precision': 0.7533718689788054, 'recall': 0.701

Seqeval tulemused
EVE {'precision': 0.5842696629213483, 'recall': 0.5473684210526316, 'f1': 0.5652173913043478, 'number': 95}
GEP {'precision': 0.6980108499095841, 'recall': 0.8230277185501066, 'f1': 0.7553816046966731, 'number': 469}
LOC {'precision': 0.811787072243346, 'recall': 0.6832, 'f1': 0.741963509991312, 'number': 625}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 8}
ORG {'precision': 0.7746288798920378, 'recall': 0.7852257181942545, 'f1': 0.7798913043478262, 'number': 731}
PER {'precision': 0.9438103599648815, 'recall': 0.9396853146853147, 'f1': 0.941743320192729, 'number': 2288}
PROD {'precision': 0.5961538461538461, 'recall': 0.5387045813586098, 'f1': 0.5659751037344398, 'number': 633}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}
overall_precision 0.8258037402815718
overall_recall 0.8101422387136673
overall_f1 0.81789802289282
overall_accuracy 0.9826683846250328

Nervaluate tulemused
Strict {'correct': 3930, 'incorrect': 707, 'partial': 0, 'mis

In [18]:
estroberta_combined_6, estroberta_combined_6_trainer, estroberta_combined_6_results = train_model('tartuNLP/EstRoBERTa', 'Combined', epochs=6)

Map:   0%|          | 0/5444 [00:00<?, ? examples/s]

Map:   0%|          | 0/833 [00:00<?, ? examples/s]

Map:   0%|          | 0/913 [00:00<?, ? examples/s]

Map:   0%|          | 0/24601 [00:00<?, ? examples/s]

Map:   0%|          | 0/3122 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/3207 [00:00<?, ? examples/s]

Kombineeritud andmestikul treenitud tartuNLP/EstRoBERTa EWT testandmestikul
Hindan testandmestikul..


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Seqeval tulemused
EVE {'precision': 0.84, 'recall': 0.7636363636363637, 'f1': 0.8000000000000002, 'number': 55}
GEP {'precision': 0.875, 'recall': 0.8166666666666667, 'f1': 0.8448275862068966, 'number': 60}
LOC {'precision': 0.3092105263157895, 'recall': 0.7580645161290323, 'f1': 0.4392523364485981, 'number': 62}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
ORG {'precision': 0.5406976744186046, 'recall': 0.5375722543352601, 'f1': 0.5391304347826088, 'number': 173}
PER {'precision': 0.9387308533916849, 'recall': 0.8469891411648569, 'f1': 0.8905033731188376, 'number': 1013}
PROD {'precision': 0.49382716049382713, 'recall': 0.5161290322580645, 'f1': 0.5047318611987381, 'number': 310}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
overall_precision 0.7470095693779905
overall_recall 0.7465630603705917
overall_f1 0.7467862481315397
overall_accuracy 0.9640153242725531

Nervaluate tulemused
Strict {'correct': 1249, 'incorrect': 250, 'partial': 0, 'missed': 174, 

Seqeval tulemused
EVE {'precision': 0.44954128440366975, 'recall': 0.5157894736842106, 'f1': 0.48039215686274517, 'number': 95}
GEP {'precision': 0.6696629213483146, 'recall': 0.6353944562899787, 'f1': 0.652078774617068, 'number': 469}
LOC {'precision': 0.6755555555555556, 'recall': 0.7296, 'f1': 0.7015384615384616, 'number': 625}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 8}
ORG {'precision': 0.7350318471337579, 'recall': 0.7893296853625171, 'f1': 0.7612137203166228, 'number': 731}
PER {'precision': 0.9456029011786038, 'recall': 0.9117132867132867, 'f1': 0.9283489096573209, 'number': 2288}
PROD {'precision': 0.6133333333333333, 'recall': 0.5813586097946287, 'f1': 0.5969180859691808, 'number': 633}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}
overall_precision 0.7946113989637306
overall_recall 0.7903525046382189
overall_f1 0.7924762298470442
overall_accuracy 0.9809466254660723

Nervaluate tulemused
Strict {'correct': 3834, 'incorrect': 854, 'partial': 0

In [20]:
estbert_combined_3, estbert_combined_3_trainer, estbert_combined_3_results = train_model('tartuNLP/EstBERT', 'Combined', epochs=3)

Map:   0%|          | 0/5444 [00:00<?, ? examples/s]

Map:   0%|          | 0/833 [00:00<?, ? examples/s]

Map:   0%|          | 0/913 [00:00<?, ? examples/s]

Map:   0%|          | 0/24601 [00:00<?, ? examples/s]

Map:   0%|          | 0/3122 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/913 [00:00<?, ? examples/s]

Map:   0%|          | 0/3207 [00:00<?, ? examples/s]

Kombineeritud andmestikul treenitud tartuNLP/EstBERT EWT testandmestikul
Hindan testandmestikul..


Seqeval tulemused
EVE {'precision': 0.49056603773584906, 'recall': 0.45614035087719296, 'f1': 0.4727272727272727, 'number': 57}
GEP {'precision': 0.3253968253968254, 'recall': 0.7454545454545455, 'f1': 0.4530386740331491, 'number': 55}
LOC {'precision': 0.37037037037037035, 'recall': 0.5454545454545454, 'f1': 0.4411764705882353, 'number': 55}
ORG {'precision': 0.7076923076923077, 'recall': 0.5974025974025974, 'f1': 0.6478873239436619, 'number': 231}
PER {'precision': 0.9563365282215123, 'recall': 0.8200913242009132, 'f1': 0.8829891838741396, 'number': 1095}
PROD {'precision': 0.43359375, 'recall': 0.30662983425414364, 'f1': 0.3592233009708738, 'number': 362}
overall_precision 0.7539393939393939
overall_recall 0.6706199460916442
overall_f1 0.7098430813124108
overall_accuracy 0.9486344417780825

Nervaluate tulemused
Strict {'correct': 1244, 'incorrect': 277, 'partial': 0, 'missed': 334, 'spurious': 149, 'possible': 1855, 'actual': 1670, 'precision': 0.7449101796407186, 'recall': 0.670619

Seqeval tulemused
EVE {'precision': 0.5555555555555556, 'recall': 0.4166666666666667, 'f1': 0.4761904761904762, 'number': 84}
GEP {'precision': 0.6988235294117647, 'recall': 0.7734375, 'f1': 0.7342398022249692, 'number': 384}
LOC {'precision': 0.8005050505050505, 'recall': 0.7188208616780045, 'f1': 0.7574671445639187, 'number': 441}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 6}
ORG {'precision': 0.7231352718078382, 'recall': 0.7846364883401921, 'f1': 0.7526315789473684, 'number': 729}
PER {'precision': 0.935226264418811, 'recall': 0.9058874086807047, 'f1': 0.9203230735647239, 'number': 2327}
PROD {'precision': 0.4976228209191759, 'recall': 0.5322033898305085, 'f1': 0.5143325143325144, 'number': 590}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}
overall_precision 0.7989035087719298
overall_recall 0.7983782599167214
overall_f1 0.7986407979831196
overall_accuracy 0.9762387122104437

Nervaluate tulemused
Strict {'correct': 3643, 'incorrect': 709, 'partial': 

In [22]:
estbert_combined_6, estbert_combined_6_trainer, estbert_combined_6_results = train_model('tartuNLP/EstBERT', 'Combined', epochs=6)

Map:   0%|          | 0/5444 [00:00<?, ? examples/s]

Map:   0%|          | 0/833 [00:00<?, ? examples/s]

Map:   0%|          | 0/913 [00:00<?, ? examples/s]

Map:   0%|          | 0/24601 [00:00<?, ? examples/s]

Map:   0%|          | 0/3122 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/3207 [00:00<?, ? examples/s]

Kombineeritud andmestikul treenitud tartuNLP/EstBERT EWT testandmestikul
Hindan testandmestikul..


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Seqeval tulemused
EVE {'precision': 0.5423728813559322, 'recall': 0.5614035087719298, 'f1': 0.5517241379310345, 'number': 57}
GEP {'precision': 0.3333333333333333, 'recall': 0.7636363636363637, 'f1': 0.46408839779005523, 'number': 55}
LOC {'precision': 0.37333333333333335, 'recall': 0.509090909090909, 'f1': 0.4307692307692308, 'number': 55}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
ORG {'precision': 0.6703910614525139, 'recall': 0.5194805194805194, 'f1': 0.5853658536585366, 'number': 231}
PER {'precision': 0.9374337221633086, 'recall': 0.8073059360730593, 'f1': 0.8675171736997056, 'number': 1095}
PROD {'precision': 0.409375, 'recall': 0.36187845303867405, 'f1': 0.38416422287390023, 'number': 362}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
overall_precision 0.7221249270286048
overall_recall 0.6668463611859838
overall_f1 0.6933856502242153
overall_accuracy 0.9462347160324535

Nervaluate tulemused
Strict {'correct': 1237, 'incorrect': 303, 'partial':

Seqeval tulemused
EVE {'precision': 0.4827586206896552, 'recall': 0.3333333333333333, 'f1': 0.39436619718309857, 'number': 84}
GEP {'precision': 0.6635514018691588, 'recall': 0.7395833333333334, 'f1': 0.6995073891625615, 'number': 384}
LOC {'precision': 0.7671568627450981, 'recall': 0.7097505668934241, 'f1': 0.7373380447585395, 'number': 441}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 6}
ORG {'precision': 0.7155612244897959, 'recall': 0.7695473251028807, 'f1': 0.7415730337078652, 'number': 729}
PER {'precision': 0.9378076062639821, 'recall': 0.9007305543618392, 'f1': 0.9188952213941254, 'number': 2327}
PROD {'precision': 0.4962962962962963, 'recall': 0.5677966101694916, 'f1': 0.5296442687747036, 'number': 590}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}
overall_precision 0.7871599564744287
overall_recall 0.7926802542187158
overall_f1 0.7899104607993012
overall_accuracy 0.9753278366705929

Nervaluate tulemused
Strict {'correct': 3617, 'incorrect': 735, 